In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from helper_functions_pipe_testing import *
from sklearn.metrics import  f1_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import *

#Instructions for the pipeline Requires four inputs:
- Mass spec data with corresponding NP surface characteristics and experimental conditions (time, concentration)
- Proteome data from uniprot (Mass, length, sequence, GO for later analysis)
- NetsurfP data for the proteins that are to be searched
- X characteristics to predict

pipeline
- Take mass spec spreadsheet Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration
- Merge with Proteome data to get file that has Accession,Enrichment,Dh,TEM,Zp,BET,Composition,Ligand,Shape,IncubationTime,IncubationConcentration,Mass,Length,and biopython features
- Merge with NSP data to get all protein features
- Split into X and Y dataset with Enrichment as label

In [ ]:
### New Data workup for RFR (Random forest regression)##

# Pull together Proteomic data
in_dir="Input_data/Proteomic data/abundance/"
#Mass Spec data input in one excel spreadsheet - Entry - Abundance labeled by NP Unique ID
#Abundance as a percent
files= os.listdir(in_dir)
for i,f in enumerate(files):
    if i==0:
        raw_MS_data=pd.read_excel(in_dir+f,header=0)
    else:
        temp = pd.read_excel(in_dir+f,header=0)
        raw_MS_data=raw_MS_data.merge(temp,how='outer',on='Entry')
# melt to make it an accession number, NPID, Abundance dataset
raw_MS_data = pd.melt(raw_MS_data, id_vars=['Entry'],var_name='NPUNID', value_name='Abundance')
#remove prots that were added due to merge
raw_MS_data=raw_MS_data.dropna()

In [ ]:
###Bring in controls (MS data for serums)##
controls=pd.read_excel('Input_data/Proteomic data/controls.xlsx',header=0)
MS_data_controls = pd.merge(raw_MS_data,controls,how='inner', on='Entry')
###Bring in Uniprot_data,NSPdata and NP data##
uniprot_filepath='UniProt/Bovine_Mouse_proteome_biopyCalcs.xlsx'
uniprot_dat=pd.read_excel(uniprot_filepath,header=0)
NSPfilePath='NetsurfP_Proteomes/Bovine_Mouse_proteome_complete.xlsx'
NSP_data=pd.read_excel(NSPfilePath)
###Bring in NP data and merge to get complete NP dataset###
NP_filepath='Input_data/NPs/NP_Database.xlsx'
NPUNdata=pd.read_excel(NP_filepath,header=0,sheet_name='NPUNID')
NPprop=pd.read_excel(NP_filepath,header=0,sheet_name='NP_Props')
NPdata=pd.merge(NPUNdata,NPprop,how="left",on='NPID')

In [ ]:
#calculate Enrichment
#####MAYBE add binning here to to keep negative results and improve capapbilities######
MS_data_controls['Enrichment']= np.log2(MS_data_controls['Abundance']/MS_data_controls['Abundance_Controls'])
MS_data=MS_data_controls.drop(columns=['Abundance','Abundance_Controls'])
raw_prop_data=pd.merge(MS_data, uniprot_dat, how='left',on='Entry')

In [ ]:
MS_data_clean = raw_MS_data.copy()
PROT_cleaned_data = normalize_mass_length_1DF(raw_prop_data) #function found in data_prep_functions line 167, normalizes mass, length and mw by dividing all values by the max in the column
Protein_data_complete = pd.merge(PROT_cleaned_data, NSP_data, left_on='Entry', right_on='Entry') #merges netsurfp features and biopython features
#creates new column called asa_sum_normalized which is the asa_sum value divide by the mass of the protein
for df in [Protein_data_complete]:
    for col in ['asa_sum']:
        df[col+'_normalized'] = df[col] / df['Mass']

data_complete= pd.merge(Protein_data_complete,NPdata,how='left', on='NPUNID')
data_complete.drop(columns=['notes','Notes','BET','NPUNID'],inplace=True)
#Optional here to drop all enrichment values that are NA - also have to deal with positive/negative infinity values
# data_complete.dropna(subset=['Enrichment'],inplace=True)

In [ ]:
data_complete.dropna(inplace=True)
data_complete= data_complete.replace([-np.inf],'-12')
data_complete=data_complete.replace([np.inf],'12')

In [ ]:
labels=data_complete['Enrichment'].copy()
labels=np.ravel(labels)
df=data_complete.drop(['Entry','Sequence','Core Material','Ligand','NPID'],axis=1)

print(labels)
# print(labels)
# print(df)
first_frame = True #starting dataframe for saving metrics
correctness_frame = pd.DataFrame()
metrics_frame = pd.DataFrame()
print_metrics = 1 #0, doesn't show metrics while runnning for each model, 1 does show metrics
trials = 100

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df,labels, test_size = 0.33, random_state=42)

rfr=RandomForestRegressor(n_estimators=100)
rfr.fit(x_train,y_train)
print(rfr.score(x_test,y_test))
print(rfr.feature_importances_)
# metrics_dict = {'AUC':metrics.roc_auc_score(y_test, rfg.predict_proba(x_test)[:, 1]),
#             'Accuracy':rfg.score(x_test, y_test), 'Recall':recall_score(y_test, rfg.predict(x_test)),
#             'Precision':precision_score(y_test, rfg.predict(x_test), zero_division=0), 'F1':f1_score(y_test, rfg.predict(x_test))}
# metrics_frame = pd.DataFrame.from_dict(data=metrics_dict,orient='index').transpose()